In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [2]:
import base64
import json
from dateutil.relativedelta import relativedelta
from datetime import datetime
import datetime

In [3]:
from typing import List

In [4]:
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic import JobState

In [5]:
from utils.vertex_ai_utils import get_custom_job_id_list
import log_manager

In [6]:
from llm.google import gemini_flash
from llm.google import gemini_pro
from llm.groq import llama3 as groq_lllama3

In [7]:
job_states = ["JOB_STATE_FAILED", "JOB_STATE_SUCCEEDED"]
job_states = ["JOB_STATE_FAILED"]

In [8]:
# pipeline_run_name = "fab-10-orion-kbl-analytics-pipeline-batch-2407092015"
pipeline_run_name = 'fab-10-orion-kbl-analytics-pipeline-batch-2406201505'
# pipeline_run_name = 'fab-10-orion-kbl-analytics-pipeline-batch-2405290940'
custom_job_id_list = get_custom_job_id_list(pipeline_run_name = pipeline_run_name,
                                          job_states = job_states)

I0000 00:00:1721185145.110687   25719 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache, work_serializer_dispatch
I0000 00:00:1721185145.145423   25719 ev_epoll1_linux.cc:125] grpc epoll fd: 56
I0000 00:00:1721185145.549042   25843 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721185146.269896   25850 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721185146.272154   25850 work_stealing_thread_pool.cc:269] WorkStealingThreadPoolImpl::Quiesce


In [9]:
job_id = custom_job_id_list[0]
job_id = 7606086089907896320
job_id = 4082988600990367744
# job_id = 3318659244651184128
# job_id = 921461662576279552

In [88]:
log_content = log_manager.get_logs_for_customjob(job_id = job_id)
print(log_content)

2024-07-09 20:35:36,275 - INFO | KBL VERTEX PIPELINE ID: 2407092015
2024-07-09 20:35:36,297 - INFO | project_id: gdw-dev-orion-streaming
2024-07-09 20:35:36,297 - INFO | fab_name: fab-10
2024-07-09 20:35:36,298 - INFO | pipeline_name: orion-kbl-analytics-pipeline
2024-07-09 20:35:36,298 - INFO | batch_id: 2407092015
2024-07-09 20:35:36,298 - INFO | BatchResultsUpdater initialized
2024-07-09 20:35:36,298 - INFO | fab_10|2407092015 AnalyticsKblJobResults initialized
2024-07-09 20:35:36,298 - INFO | fab_10|2407092015 AnalyticsKblJobSamples initialized
2024-07-09 20:35:36,298 - INFO | AnalyticsKblBatchJobStage initialized
2024-07-09 20:35:36,299 - INFO | AnalyticsKblBatchJob initialized
2024-07-09 20:35:36,299 - INFO | AnalyticsKblBatchJobTrigger initialized
2024-07-09 20:35:36,300 - INFO | orion-kbl-analytics-pipeline|fab-10|2407092015|ResultUpdater|execution started
2024-07-09 20:35:36,300 - INFO | orion-kbl-analytics-pipeline|fab-10|2407092015|ResultUpdater|fab-10 Batch_Id:2407092015 up

In [89]:
from llm.google import gemini_flash
from llm.google import gemini_pro
from llm.groq import llama3 as groq_lllama3

In [94]:
prompt  = f"""You are a experienced in Application Log Analyser.
Each log entries are ordered in sequencial manner.
You will be provided the log entries by triple backticks
```
{log_content} 
```
Ignore any warnings and any deprecated messages from log entries
Give a binary score 'yes' or 'no' score only to indicate whether log entries have issues/exception/error
"""
# print(prompt)
result = gemini_flash.invoke(prompt).content
print(result)

no 



In [95]:
# What is key error, exception with message, file_name in json format only

In [97]:
prompt  = f"""You are a experienced Google Cloud Log Analyser from application.
You will be provided the log entries by triple backticks
```
{log_content} 
```
Strictly Ignore any warning and deprecated messages from log entries
What is key error, exception with message in json format only
Your response must be a JSON object has the following schema:
* exception: exception message from log entries
* info: description of key error 
* file_name : only name of file where have issues
"""
# print(prompt)
result = gemini_flash.invoke(prompt).content
print(result)

```json
{
  "exception": null,
  "info": "No key error found in the provided logs.",
  "file_name": null
}
```


In [99]:
prompt  = f"""You are a experienced Google Cloud Log Analysis Expert.
Each log entries are ordered in sequencial manner.
You will be provided the log entries by triple backticks
```
{log_content} 
```
Strictly Ignore any warning and deprecated messages from log entries
Extract all the python Traceback only if ant key error or exception without duplicate from orion_kbl_analytics_lib in file modules only
"""
# print(prompt)
result = gemini_flash.invoke(prompt).content
print(result)

```python
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/orion_kbl_analytics_lib/analytics/job_results_utils.py", line 132, in <module>
    df_results_formatted['p_value'] = df_results_formatted['p_value'].round(36).astype(pd.ArrowDtype(pa.decimal256(56, 36)))
  File "/usr/local/lib/python3.10/site-packages/pandas/core/frame.py", line 3817, in __getitem__
    indexer = self._get_indexer_for_level(key, axis=axis)
  File "/usr/local/lib/python3.10/site-packages/pandas/core/frame.py", line 4021, in _get_indexer_for_level
    return self.columns.get_loc(key)
  File "/usr/local/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3127, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 157, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 177, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 204, in pandas._libs.index.IndexEngine.get_loc
  File "pan

In [101]:
prompt  = f"""You are a experienced Google Cloud Log Analyser from application.
Each log entries are ordered in sequencial manner.
You will be provided the log entries by triple backticks
```
{log_content} 
```
Strictly Ignore any warning from log entries
Give a binary score 'yes' or 'no' score only to indicate whether log entries contains completed SQL Query Statement from log entries
"""
# print(prompt)
result = gemini_flash.invoke(prompt).content
print(result)

if result == 'yes':
    prompt  = f"""You are a experienced Google Cloud Log Analyser from application.
    Each log entries are ordered in sequencial manner.
    You will be provided the log entries by triple backticks
    ```
    {log_content} 
    ```
    Strictly Ignore any warning from log entries
    Extract all SQL Query used from logs in JSON FORMAT
    """
    # print(prompt)
    result = gemini_flash.invoke(prompt).content
    print(result)


yes 

